In [69]:
import pandas as pd
import numpy as np

In [71]:
# load data
df = pd.read_csv("../data/combine.csv")
df.head()
len(df.columns)

28

In [45]:
# define to check if close price increased compare to the previous close
df["trend"] = df.apply(lambda x: 1 if x["btc_diff"] > 0 else (-1 if x["btc_diff"] <0 else 0), axis=1)

In [46]:
df[["date", "trend"]]

,date,trend
0,2012-01-03,0
1,2012-01-04,1
2,2012-01-05,1
3,2012-01-06,1
4,2012-01-09,0
...,...,...
2318,2021-03-25,-1
2319,2021-03-26,1
2320,2021-03-29,1
2321,2021-03-30,1


In [47]:
ind_train_start = 0
ind_test_start = int((len(df) - 0) * 0.8)

In [67]:
df.columns

Index(['date', 'close', 'ma_5', 'ma_10', 'ma_20', 'ma_30', 'ma_60', 'ma_90',
       'ma_180', 'ma_240', 'ma_360', 'gold', 'nasdaq_comp', 'sp500', 'indu',
       'oil', 'btc_diff', 'gold_diff', 'nasdaq_diff', 'sp500_diff',
       'indu_diff', 'oil_diff', 'btc_diffpct', 'gold_diffpct',
       'nasdaq_diffpct', 'sp500_diffpct', 'indu_diffpct', 'oil_diffpct',
       'trend'],
      dtype='object')

In [49]:
# change here
X = df[['gold', 'nasdaq_comp', 'sp500', 'indu', 'oil']]
y = df["trend"]

In [50]:
X_train = X.iloc[ind_train_start:ind_test_start]
X_test = X.iloc[ind_test_start:]
X_test = X_test.iloc[:len(X_test)-1]
X_test.iloc[0]

# for using close price
#y_train = y.iloc[ind_train_start + 1:ind_test_start + 1]
#y_test = y.iloc[ind_test_start+1:]
# for using other rows
y_train = y.iloc[ind_train_start+1:ind_test_start+1]
y_test = y.iloc[ind_test_start+1:]

In [51]:
print(X_train.shape, y_train.shape)

(1858, 5) (1858,)


In [52]:
ind_test = ind_test_start + 3
ind_test2 = ind_test - ind_test_start
print(df[["date", "close", 'gold', 'nasdaq_comp', 'sp500', 'indu', 'oil', "btc_diff", "trend"]].iloc[ind_test])
print(X_test.iloc[ind_test2])
print(y_test.iloc[ind_test2])

date           2019-06-03
close             8741.79
gold                13.09
nasdaq_comp       7333.02
sp500             2744.45
indu             24819.78
oil                 16.11
btc_diff           167.04
trend                   1
Name: 1861, dtype: object
gold              13.09
nasdaq_comp     7333.02
sp500           2744.45
indu           24819.78
oil               16.11
Name: 1861, dtype: float64
-1


In [53]:
from sklearn.preprocessing import MinMaxScaler

In [54]:
X_scaler = MinMaxScaler().fit(X)
# for predict price
# Y_scaler = MinMaxScaler().fit(y)

In [55]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [56]:
# for predict price
#y_train_scaled = Y_scaler.transform(y_train)
#y_test_scaled = Y_scaler.transform(y_test)

In [57]:
#y_train_scaled_shape = np.reshape(y_train_scaled, (y_train_scaled.shape[0]))
#print(y_train_scaled_shape.shape)

In [58]:
print(X_train_scaled, y_train)

[[9.47140579e-01 3.14489858e-05 0.00000000e+00 1.40446732e-02
  9.01944170e-01]
 [9.63317384e-01 0.00000000e+00 8.89719294e-05 1.50432536e-02
  9.23666008e-01]
 [9.58760538e-01 1.87820332e-03 1.48286549e-03 1.49141596e-02
  9.23666008e-01]
 ...
 [1.37161085e-01 4.35037315e-01 5.72823524e-01 6.35456440e-01
  2.51368577e-01]
 [1.36933242e-01 4.35799953e-01 5.74239661e-01 6.39975681e-01
  2.51368577e-01]
 [1.32148553e-01 4.33208906e-01 5.65464804e-01 6.28683749e-01
  2.60034585e-01]] 1       1
2       1
3       1
4       0
5       1
       ..
1854   -1
1855   -1
1856    1
1857    1
1858   -1
Name: trend, Length: 1858, dtype: int64


In [59]:
print(X_train_scaled.shape, y_train.shape)

(1858, 5) (1858,)


In [60]:
y_train_reshape = np.reshape(y_train_scaled, (y_train_scaled.shape[0]))
y_test_reshape = np.reshape(y_test_scaled, (y_test_scaled.shape[0]))

NameError: name 'y_train_scaled' is not defined

In [61]:
from sklearn.svm import SVC 
model = SVC(kernel='linear')
model

SVC(kernel='linear')

In [62]:
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50, 100, 1000],
              'gamma': [0.00001, 0.0005, 0.0001, 0.0005, 0.0001]}
grid = GridSearchCV(model, param_grid, verbose=3)

In [63]:
grid.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 60 candidates, totalling 300 fits
[CV 1/5] END ...........................C=0.001, gamma=1e-05; total time=   0.0s
[CV 2/5] END ...........................C=0.001, gamma=1e-05; total time=   0.0s
[CV 3/5] END ...........................C=0.001, gamma=1e-05; total time=   0.0s
[CV 4/5] END ...........................C=0.001, gamma=1e-05; total time=   0.0s
[CV 5/5] END ...........................C=0.001, gamma=1e-05; total time=   0.0s
[CV 1/5] END ..........................C=0.001, gamma=0.0005; total time=   0.0s
[CV 2/5] END ..........................C=0.001, gamma=0.0005; total time=   0.0s
[CV 3/5] END ..........................C=0.001, gamma=0.0005; total time=   0.0s
[CV 4/5] END ..........................C=0.001, gamma=0.0005; total time=   0.0s
[CV 5/5] END ..........................C=0.001, gamma=0.0005; total time=   0.0s
[CV 1/5] END ..........................C=0.001, gamma=0.0001; total time=   0.0s
[CV 2/5] END ..........................C=0.001,

[CV 2/5] END .............................C=0.1, gamma=1e-05; total time=   0.0s
[CV 3/5] END .............................C=0.1, gamma=1e-05; total time=   0.0s
[CV 4/5] END .............................C=0.1, gamma=1e-05; total time=   0.0s
[CV 5/5] END .............................C=0.1, gamma=1e-05; total time=   0.0s
[CV 1/5] END ............................C=0.1, gamma=0.0005; total time=   0.0s
[CV 2/5] END ............................C=0.1, gamma=0.0005; total time=   0.0s
[CV 3/5] END ............................C=0.1, gamma=0.0005; total time=   0.0s
[CV 4/5] END ............................C=0.1, gamma=0.0005; total time=   0.0s
[CV 5/5] END ............................C=0.1, gamma=0.0005; total time=   0.0s
[CV 1/5] END ............................C=0.1, gamma=0.0001; total time=   0.0s
[CV 2/5] END ............................C=0.1, gamma=0.0001; total time=   0.0s
[CV 3/5] END ............................C=0.1, gamma=0.0001; total time=   0.0s
[CV 4/5] END ...............

[CV 5/5] END ..............................C=10, gamma=1e-05; total time=   0.0s
[CV 1/5] END .............................C=10, gamma=0.0005; total time=   0.0s
[CV 2/5] END .............................C=10, gamma=0.0005; total time=   0.0s
[CV 3/5] END .............................C=10, gamma=0.0005; total time=   0.0s
[CV 4/5] END .............................C=10, gamma=0.0005; total time=   0.0s
[CV 5/5] END .............................C=10, gamma=0.0005; total time=   0.0s
[CV 1/5] END .............................C=10, gamma=0.0001; total time=   0.0s
[CV 2/5] END .............................C=10, gamma=0.0001; total time=   0.0s
[CV 3/5] END .............................C=10, gamma=0.0001; total time=   0.0s
[CV 4/5] END .............................C=10, gamma=0.0001; total time=   0.0s
[CV 5/5] END .............................C=10, gamma=0.0001; total time=   0.0s
[CV 1/5] END .............................C=10, gamma=0.0005; total time=   0.0s
[CV 2/5] END ...............

GridSearchCV(estimator=SVC(kernel='linear'),
             param_grid={'C': [0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50,
                               100, 1000],
                         'gamma': [1e-05, 0.0005, 0.0001, 0.0005, 0.0001]},
             verbose=3)

In [64]:
print(grid.best_params_)
print(grid.best_score_)

{'C': 0.5, 'gamma': 1e-05}
0.5295177230965423


In [66]:
grid.score(X_test_scaled, y_test)

0.5107758620689655

In [ ]:
grid.save("svc.h5")